In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
plm_layers = list(range(1,13))
plm_vp = [330, 1480, 1928.7, 2089.9, 2111.5, 2451.9, 2606.4, 2990.7, 2910.7, 3429.3, 3800, 4300]
plm_depth = [0.0, 150.0, 482.1, 970.1, 1473.6, 1972.0, 2465.2, 2968.8, 3462.0, 3960.4, 4500.0, 5000.0]

In [3]:
df_plm = pd.DataFrame(list(zip(plm_layers, plm_depth, plm_vp)), columns=['Intf', 'Depth', 'Vp'])

In [4]:
df_plm

,Intf,Depth,Vp
0,1,0.0,330.0
1,2,150.0,1480.0
2,3,482.1,1928.7
3,4,970.1,2089.9
4,5,1473.6,2111.5
5,6,1972.0,2451.9
6,7,2465.2,2606.4
7,8,2968.8,2990.7
8,9,3462.0,2910.7
9,10,3960.4,3429.3


In [5]:
df_plm['Thickness'] = df_plm.diff()[['Depth']]
df_plm['dOWT'] = df_plm['Thickness'] / df_plm['Vp']
df_plm['OWT'] = df_plm.cumsum()['dOWT']
df_plm['TWT'] = df_plm['OWT']*2

In [6]:
df_plm

,Intf,Depth,Vp,Thickness,dOWT,OWT,TWT
0,1,0.0,330.0,NaN,NaN,NaN,NaN
1,2,150.0,1480.0,150.0,0.101351,0.101351,0.202703
2,3,482.1,1928.7,332.1,0.172189,0.273540,0.547080
3,4,970.1,2089.9,488.0,0.233504,0.507044,1.014088
4,5,1473.6,2111.5,503.5,0.238456,0.745500,1.491000
5,6,1972.0,2451.9,498.4,0.203271,0.948771,1.897542
6,7,2465.2,2606.4,493.2,0.189227,1.137997,2.275995
7,8,2968.8,2990.7,503.6,0.168389,1.306386,2.612772
8,9,3462.0,2910.7,493.2,0.169444,1.475830,2.951660
9,10,3960.4,3429.3,498.4,0.145336,1.621166,3.242331


In [7]:
def compute_mute_offset(model, percmute):
    Smax = 1 + percmute/100
    muteangle = np.arccos(1/Smax)
    muteangledeg = np.round(np.degrees(muteangle))
    print('Mute angle is: ' + str(muteangledeg))
    model['Mute angle - rad'] = muteangle
    model['Mute angle - deg'] = muteangledeg
    model['Constant'] = np.sin(muteangle) / model['Vp']
    intf_list = list(model['Intf'])[1:]
    vp_list = list(model['Vp'])[1:]
    depth_list = list(model['Thickness'])[1:]
    constant = list(model['Constant'])[1:]
    twt_list = list(model['TWT'])[1:]
    mute_onset = [np.nan]
    for i in range(len(intf_list)):
        const = constant[i]
        print('i is: ' + str(i))
        print('Target is: ' + str(intf_list[i]))
        print('TWT is: ' + str(twt_list[i]))
        x = []
        for j, intf in enumerate(intf_list[:i]):        
            print(i, j, intf, vp_list[j])
            alpha = np.arcsin(const * vp_list[j])
            x.append(depth_list[j] * np.tan(alpha))
            print('alpha = ' + str(np.degrees(alpha)))
        x.append(depth_list[i] * np.tan(muteangle))
        print('x = ' + str(x))
        x_sum = sum(x) * 2
        print('x_sum is: ' + str(x_sum) + ' trace no: ' + str(x_sum/12.5))
        print('----')
        mute_onset.append(x_sum)
    coltitle = str(percmute) + '% /' + str(muteangledeg)  + ' deg mute offset'
    coltitlechan = str(percmute) + '% /' + str(muteangledeg)  + ' deg mute chan no'
    model[coltitle] = mute_onset
    model[coltitlechan] = model[coltitle] / 12.5
    
    return model

In [8]:
test = compute_mute_offset(df_plm, 41.42)

Mute angle is: 45.0
i is: 0
Target is: 2
TWT is: 0.20270270270270271
x = [149.99712297240907]
x_sum is: 299.99424594481815 trace no: 23.999539675585453
----
i is: 1
Target is: 3
TWT is: 0.5470797442332673
1 0 2 1480.0
alpha = 32.86064472522725
x = [96.89326431573133, 332.0936302609137]
x_sum is: 857.97378915329 trace no: 68.6379031322632
----
i is: 2
Target is: 4
TWT is: 1.0140877350462247
2 0 2 1480.0
alpha = 30.049333497865998
2 1 3 1928.7
alpha = 40.73484441483929
x = [86.77483246421063, 286.00263623817784, 487.99064007023753]
x_sum is: 1721.536217545252 trace no: 137.72289740362015
----
i is: 3
Target is: 5
TWT is: 1.490999882808479
3 0 2 1480.0
alpha = 29.71084115807795
3 1 3 1928.7
alpha = 40.231976620937814
3 2 4 2089.9
alpha = 44.416300905689546
x = [85.59610955738515, 280.96411394403117, 478.1569633977666, 503.49034277738633]
x_sum is: 2696.4150593531385 trace no: 215.7132047482511
----
i is: 4
Target is: 6
TWT is: 1.8975417483005463
4 0 2 1480.0
alpha = 25.265608969039878
4 1

In [9]:
test = compute_mute_offset(df_plm, 50)

Mute angle is: 48.0
i is: 0
Target is: 2
TWT is: 0.20270270270270271
x = [167.70509831248427]
x_sum is: 335.41019662496853 trace no: 26.83281572999748
----
i is: 1
Target is: 3
TWT is: 0.5470797442332673
1 0 2 1480.0
alpha = 34.88656745472597
x = [104.58917760817701, 371.29908766384017]
x_sum is: 951.7765305440344 trace no: 76.14212244352275
----
i is: 2
Target is: 4
TWT is: 1.0140877350462247
2 0 2 1480.0
alpha = 31.8594362062531
2 1 3 1928.7
alpha = 43.461305540844606
x = [93.21950180234722, 314.72514944212804, 545.6005865099488]
x_sum is: 1907.0904755088482 trace no: 152.56723804070785
----
i is: 3
Target is: 5
TWT is: 1.490999882808479
3 0 2 1480.0
alpha = 31.495899447865234
3 1 3 1928.7
alpha = 42.90837169138622
3 2 4 2089.9
alpha = 47.53850797968584
x = [91.9053523066163, 308.696802435226, 533.277666176322, 562.9301133355721]
x_sum is: 2993.619868507473 trace no: 239.48958948059783
----
i is: 4
Target is: 6
TWT is: 1.8975417483005463
4 0 2 1480.0
alpha = 26.73771459236227
4 1 3 1

In [10]:
test = compute_mute_offset(df_plm, 100)

Mute angle is: 60.0
i is: 0
Target is: 2
TWT is: 0.20270270270270271
x = [259.80762113533166]
x_sum is: 519.6152422706633 trace no: 41.56921938165306
----
i is: 1
Target is: 3
TWT is: 0.5470797442332673
1 0 2 1480.0
alpha = 41.64781237897708
x = [133.4002392234129, 575.2140731936244]
x_sum is: 1417.2286248340745 trace no: 113.37828998672596
----
i is: 2
Target is: 4
TWT is: 1.0140877350462247
2 0 2 1480.0
alpha = 37.82787156668919
2 1 3 1928.7
alpha = 53.05628976100373
x = [116.46884141860022, 441.61360557037176, 845.2407940936123]
x_sum is: 2806.6464821651684 trace no: 224.53171857321348
----
i is: 3
Target is: 5
TWT is: 1.490999882808479
3 0 2 1480.0
alpha = 37.37416481905577
3 1 3 1928.7
alpha = 52.28379390173179
3 2 4 2089.9
alpha = 58.9998807165904
x = [114.57652350097945, 429.4360967296876, 812.1645574021923, 872.0875816109296]
x_sum is: 4456.529518487578 trace no: 356.52236147900624
----
i is: 4
Target is: 6
TWT is: 1.8975417483005463
4 0 2 1480.0
alpha = 31.51653765391006
4 1 3

In [11]:
test.drop(['Mute angle - rad', 'Mute angle - deg','Constant'], axis=1)

,Intf,Depth,Vp,Thickness,dOWT,OWT,TWT,41.42% /45.0 deg mute offset,41.42% /45.0 deg mute chan no,50% /48.0 deg mute offset,50% /48.0 deg mute chan no,100% /60.0 deg mute offset,100% /60.0 deg mute chan no
0,1,0.0,330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,150.0,1480.0,150.0,0.101351,0.101351,0.202703,299.994246,23.999540,335.410197,26.832816,519.615242,41.569219
2,3,482.1,1928.7,332.1,0.172189,0.273540,0.547080,857.973789,68.637903,951.776531,76.142122,1417.228625,113.378290
3,4,970.1,2089.9,488.0,0.233504,0.507044,1.014088,1721.536218,137.722897,1907.090476,152.567238,2806.646482,224.531719
4,5,1473.6,2111.5,503.5,0.238456,0.745500,1.491000,2696.415059,215.713205,2993.619869,239.489589,4456.529518,356.522361
5,6,1972.0,2451.9,498.4,0.203271,0.948771,1.897542,3093.136246,247.450900,3392.170251,271.373620,4723.821608,377.905729
6,7,2465.2,2606.4,493.2,0.189227,1.137997,2.275995,3789.269592,303.141567,4152.793884,332.223511,5766.100638,461.288051
7,8,2968.8,2990.7,503.6,0.168389,1.306386,2.612772,4075.477068,326.038165,4440.182891,355.214631,5993.929971,479.514398
8,9,3462.0,2910.7,493.2,0.169444,1.475830,2.951660,5242.180469,419.374438,5756.317967,460.505437,8148.000664,651.840053
9,10,3960.4,3429.3,498.4,0.145336,1.621166,3.242331,5075.688727,406.055098,5514.078773,441.126302,7337.485824,586.998866
